In [25]:
!unzip competitive-data-science-predict-future-sales.zip

Archive:  competitive-data-science-predict-future-sales.zip
replace test.csv.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.csv.gz             
  inflating: sample_submission.csv.gz  
  inflating: sales_train.csv.gz      
  inflating: shops.csv               
  inflating: item_categories.csv     
  inflating: items.csv               


In [0]:
import numpy as np 
import pandas as pd 

In [0]:
'''
loading data
'''
sales = pd.read_csv('sales_train.csv.gz', parse_dates=['date'], infer_datetime_format=True, dayfirst=True)
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
cats = pd.read_csv('item_categories.csv')
test_df = pd.read_csv('test.csv.gz')

In [0]:
'''
rearange to be monthly item count by shop
'''
# item_cnt_day -> item_cnt_month
df = sales.groupby([sales.date.apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).sum().reset_index()
df = df[['date', 'item_id', 'shop_id', 'item_cnt_day']]
df.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)

In [0]:
df["item_cnt_month"].clip(0.,20.,inplace=True)
df = df.pivot_table(index=['item_id','shop_id'], columns='date',values='item_cnt_month',fill_value=0).reset_index()

In [0]:
# merge df from monthly item count to specific item-shop in test_data
test = pd.merge(test_df,df,on=['item_id','shop_id'], how='left').fillna(0)

In [0]:
test.drop(['ID', 'item_id', 'shop_id'], axis=1, inplace=True)

In [0]:
'''
rearange to be monthly average price by shop
'''
# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
sales["item_price"] = scaler.fit_transform(sales["item_price"].values.reshape(-1,1))

In [0]:
df2 = sales.groupby([sales.date.apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).mean().reset_index()
df2 = df2[['date','item_id','shop_id','item_price']].pivot_table(index=['item_id','shop_id'], columns='date',values='item_price',fill_value=0).reset_index()

In [0]:
# merge df2 from monthly average price to specific item-shop in test_data
price = pd.merge(test_df,df2,on=['item_id','shop_id'], how='left').fillna(0)
price = price.drop(labels=['ID','item_id','shop_id'],axis=1)

In [0]:
'''
create train & test data set
'''
# train data
Y_train = test['2015-10']
X_sales = test.drop(['2015-10'], axis=1)
X_sales = X_sales.values.reshape((X_sales.shape[0], X_sales.shape[1], 1))
X_prices = price.drop(['2015-10'], axis=1)
X_prices = X_prices.values.reshape((X_prices.shape[0], X_prices.shape[1], 1))

In [0]:
X = np.append(X_sales, X_prices, axis=2)
Y = Y_train.values.reshape((Y_train.shape[0], 1))

In [0]:
# test data
X_test_sales = test.drop(['2013-01'], axis = 1)
X_test_sales = X_test_sales.values.reshape((X_test_sales.shape[0], X_test_sales.shape[1], 1))
X_test_prices = price.drop(['2013-01'], axis=1)
X_test_prices = X_test_prices.values.reshape((X_test_prices.shape[0], X_test_prices.shape[1], 1))

In [0]:
test = np.append(X_test_sales, X_test_prices, axis=2)

In [72]:
'''
create model
'''
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(LSTM(16, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
